# Getting Started

The purpose of the present **Getting Started** section is to give a quick overview
of the main objects and features of the LArray library.
To get a more detailed presentation of all capabilities of LArray, read the next sections of the tutorial.

The [API Reference](../api.rst#api-reference) section of the documentation give you the list of all objects, methods and functions with their individual documentation and examples.

To use the LArray library, the first thing to do is to import it:

In [ ]:
%xmode Minimal

In [ ]:
from larray import *

To know the version of the LArray library installed on your machine, type:

In [ ]:
from larray import __version__
__version__

<div class="alert alert-warning">

**Warning:** 
The tutorial is generated from Jupyter notebooks which work in the "interactive" mode (like in the LArray Editor console). 
In the interactive mode, there is no need to use the print() function to display the content of a variable.
Simply writing its name is enough.
The same remark applies for the returned value of an expression.<br><br>
In a Python script (file with .py extension), you always need to use the print() function to display the content of 
a variable or the value returned by a function or an expression.
    
</div>

In [ ]:
s = 1 + 2

# In the interactive mode, there is no need to use the print() function 
# to display the content of the variable 's'.
# Simply typing 's' is enough
s

In [ ]:
# In the interactive mode, there is no need to use the print() function 
# to display the result of an expression
1 + 2

## Create an array

Working with the LArray library mainly consists of manipulating [Array](../api.rst#array) data structures.
They represent N-dimensional labelled arrays and are composed of raw data (NumPy ndarray), axes and optionally some metadata.

An [Axis](../api.rst#axis) object represents a dimension of an array. 
It contains a list of labels and has a name.
They are several ways to create an axis:

In [ ]:
# create an axis using one string
age = Axis('age=0-9,10-17,18-66,67+')
# labels generated using the special syntax start..end
time = Axis('time=2015..2017')
# labels given as a list
gender = Axis(['female', 'male'], 'gender')

age, gender, time

<div class="alert alert-warning">

**Warning:** When using the string syntax `"axis_name=list,of,labels"` or `"axis_name=start..end"`, LArray will automatically infer the type of labels. For example, `age = Axis("age=0..100")` will create an age axis with labels of type `int`.<br><br>
  Mixing numbers with letters or special characters like `+` will create an axis with labels of type `str` instead of `int`.<br>
  For example, `age = Axis("age=0..98,99+")` will create an age axis with labels of type `str` instead of `int`! 

</div>

The labels allow to select subsets and to manipulate the data without working with the positions
of array elements directly.

To create an array from scratch, you need to supply data and axes:

In [ ]:
# define some data. This is the belgian population (in thousands). Source: eurostat.
data = [[[633, 635, 634],
         [663, 665, 664]],
        [[484, 486, 491],
         [505, 511, 516]],
        [[3572, 3581, 3583],
         [3600, 3618, 3616]],
        [[1023, 1038, 1053],
         [756, 775, 793]]]

# create an Array object
population = Array(data, axes=[age, gender, time])
population

You can optionally attach some metadata to an array:

In [ ]:
# attach some metadata to the population array
population.meta.title = 'population by age, gender and year'
population.meta.source = 'Eurostat'

# display metadata
population.meta

To get a short summary of an array, type:

In [ ]:
# Array summary: metadata + dimensions + description of axes
population.info

To get the axes of an array, type:

In [ ]:
population.axes

It is also possible to extract one axis belonging to an array using its name:

In [ ]:
# extract the 'time' axis belonging to the 'population' array
time = population.time
time

## Create an array filled with predefined values

Arrays filled with predefined values can be generated through [dedicated functions](../api.rst#array-creation-functions):

  - `zeros` : creates an array filled with 0
  - `ones` : creates an array filled with 1
  - `full` : creates an array filled with a given value
  - `sequence` : creates an array by sequentially applying modifications to the array along axis.
  - `ndtest` : creates a test array with increasing numbers as data

In [ ]:
zeros([age, gender])

In [ ]:
ones([age, gender])

In [ ]:
full([age, gender], fill_value=10.0)

In [ ]:
# With initial=1.0 and inc=0.5, we generate the sequence 1.0, 1.5, 2.0, 2.5, 3.0, ...
sequence(age, initial=1.0, inc=0.5)

In [ ]:
ndtest([age, gender])

## Save/Load an array

The LArray library offers many I/O functions to read and write arrays in various formats
(CSV, Excel, HDF5). For example, to save an array in a CSV file, call the method `to_csv`:

In [ ]:
# save our population array to a CSV file
population.to_csv('population_belgium.csv')

The content of the CSV file is then:

    age,gender\time,2015,2016,2017
    0-9,female,633,635,634
    0-9,male,663,665,664
    10-17,female,484,486,491
    10-17,male,505,511,516
    18-66,female,3572,3581,3583
    18-66,male,3600,3618,3616
    67+,female,1023,1038,1053
    67+,male,756,775,793  

<div class="alert alert-info">

**Note:** In CSV or Excel files, the last dimension is horizontal and the names of the
last two dimensions are separated by a backslash \.

</div>

To load a saved array, call the function `read_csv`:

In [ ]:
population = read_csv('population_belgium.csv')
population

Other input/output functions are described in the [Input/Output](../api.rst#input-output) section of the API documentation.

## Selecting a subset

To select an element or a subset of an array, use brackets [ ].
In Python we usually use the term *indexing* for this operation.

Let us start by selecting a single element:

In [ ]:
population['67+', 'female', 2017]

Labels can be given in arbitrary order:

In [ ]:
population[2017, 'female', '67+']

When selecting a larger subset the result is an array:

In [ ]:
population['female']

When selecting several labels for the same axis, they must be given as a list (enclosed by ``[ ]``)

In [ ]:
population['female', ['0-9', '10-17']]

You can also select *slices*, which are all labels between two bounds (we usually call them the `start` and `stop`
bounds). Specifying the `start` and `stop` bounds of a slice is optional: when not given, `start` is the first label
of the corresponding axis, `stop` the last one:

In [ ]:
# in this case '10-17':'67+' is equivalent to ['10-17', '18-66', '67+']
population['female', '10-17':'67+']

In [ ]:
# :'18-66' selects all labels between the first one and '18-66'
# 2017: selects all labels between 2017 and the last one
population[:'18-66', 2017:]

<div class="alert alert-info">

**Note:** Contrary to slices on normal Python lists, the stop bound is included in the selection.

</div>

<div class="alert alert-warning">

**Warning:** Selecting by labels as above only works as long as there is no ambiguity.
When several axes have some labels in common and you do not specify explicitly
on which axis to work, it fails with an error ending with something like:
`ValueError: <somelabel> is ambiguous (valid in <axis1>, <axis2>)`

</div>

For example, imagine you need to work with an 'immigration' array containing two axes sharing some common labels:

In [ ]:
country = Axis(['Belgium', 'Netherlands', 'Germany'], 'country')
citizenship = Axis(['Belgium', 'Netherlands', 'Germany'], 'citizenship')

immigration = ndtest((country, citizenship, time))

immigration

If we try to get the number of Belgians living in the Netherlands for the year 2017, we might try something like:

```python
    immigration['Netherlands', 'Belgium', 2017]
```

... but we receive back a volley of insults:

```
[some long error message ending with the line below]
[...]
ValueError: Netherlands is ambiguous (valid in country, citizenship)
```

In that case, we have to specify explicitly which axes the 'Netherlands' and 'Belgium' labels we want to select belong to:

In [ ]:
immigration[country['Netherlands'], citizenship['Belgium'], 2017]

## Aggregation

The LArray library includes many [aggregations methods](../api.rst#aggregation-functions): sum, mean, min, max, std, var, ...

For example, assuming we still have an array in the ``population`` variable:

In [ ]:
population

We can sum along the 'gender' axis using:

In [ ]:
population.sum(gender)

Or sum along both 'age' and 'gender':

In [ ]:
population.sum(age, gender)

It is sometimes more convenient to aggregate along all axes **except** some. In that case, use the aggregation
methods ending with `_by`. For example:

In [ ]:
population.sum_by(time)

## Groups

A [Group](../api.rst#group) object represents a subset of labels or positions of an axis:

In [ ]:
children = age['0-9', '10-17']
children

It is often useful to attach them an explicit name using the ``>>`` operator:

In [ ]:
working = age['18-66'] >> 'working'
working

In [ ]:
nonworking = age['0-9', '10-17', '67+'] >> 'nonworking'
nonworking

Still using the same ``population`` array:

In [ ]:
population

Groups can be used in selections:

In [ ]:
population[working]

In [ ]:
population[nonworking]

or aggregations:

In [ ]:
population.sum(nonworking)

When aggregating several groups, the names we set above using ``>>`` determines the label on the aggregated axis.
Since we did not give a name for the children group, the resulting label is generated automatically :

In [ ]:
population.sum((children, working, nonworking))

<div class="alert alert-warning">

**Warning:** Mixing slices and individual labels inside the `[ ]` will generate **several groups** (a tuple of groups) instead of a single group.<br>
    If you want to create a single group using both slices and individual labels, you need to use the `.union()` method (see below).   
  
</div>

In [ ]:
age_100 = Axis('age=0..100')

# mixing slices and individual labels leads to the creation of several groups (a tuple of groups)
age_100[0:10, 20, 30, 40]

In [ ]:
# the union() method allows to mix slices and individual labels to create a single group
age_100[0:10].union(age_100[20, 30, 40])

## Grouping arrays in a Session

Variables (arrays) may be grouped in [Session](../api.rst#session) objects.
A session is an ordered dict-like container with special I/O methods:

In [ ]:
population = zeros([age, gender, time])
births = zeros([age, gender, time])
deaths = zeros([age, gender, time])

# create a session containing the arrays of the model
demography_session = Session(population=population, births=births, deaths=deaths)

# get an array (option 1)
demography_session['population']

# get an array (option 2)
demography_session.births

# modify an array
demography_session.deaths['male'] = 1

# add an array
demography_session.foreigners = zeros([age, gender, time])

# displays names of arrays contained in the session
# (in alphabetical order)
demography_session.names

One of the main interests of using sessions is to save and load many arrays at once:

In [ ]:
# dump all arrays contained in demography_session in one HDF5 file
demography_session.save('demography.h5')
# load all arrays saved in the HDF5 file 'demography.h5' and store them in the 'demography_session' variable
demography_session = Session('demography.h5')

However, development tools like PyCharm do not provide *autocomplete* for objects in ``Session`` objects.

*Autocomplete* is the feature in which development tools try to predict the variable or function a user intends to enter after only a few characters have been typed (like word completion in cell phones).

Another way to group objects of a model is to use [CheckedSession](../api.rst#checkedsession). 
The ``CheckedSession`` provide the same methods than ``Session`` but enable the *autocomplete* feature on objects it contains.

For more details about ``Session`` and ``CheckedSession``, see the [Working With Sessions](tutorial_sessions.ipynb#Working-With-Sessions) section of the tutorial.

To get the list of methods belonging to the ``Session`` and ``CheckedSession`` ojects, check the [corresponding section](../api.rst#session) in the API Reference.

## Graphical User Interface (Editor)

The LArray project provides an optional package called [larray-editor](../api.rst#editor) allowing users to explore and edit arrays through a graphical interface.

The `view()` function displays the content of (an) array(s) in a graphical user interface in read-only mode. 

For instance, the statement
```python
view(population)
```
will open a new window showing the values and axes of the 'population' array. 

The statement
```python
view(demography_session)
```
will show all arrays contained in the 'demography_session'.

A session can be directly loaded from a file
```python
view('demography.h5')
```

Calling
```python
view()
```
with no passed argument creates a session with all existing arrays from the current namespace and shows its content.

<div class="alert alert-info">

**Notes**: 

  - Calling `view()` will block the execution of the rest of code until the graphical user interface is closed!
  - The larray-editor tool is automatically available when installing the **larrayenv** metapackage from conda.

</div>

To open the user interface in edit mode, call the `edit()` function instead.

![compare](../_static/editor.png)

Finally, you can also visually compare two arrays or sessions using the `compare()` function:

```python
    arr0 = ndtest((3, 3))
    arr1 = ndtest((3, 3))
    arr1[['a1', 'a2']] = -arr1[['a1', 'a2']]
    compare(arr0, arr1)
```

![compare](../_static/compare.png)

### For Windows Users

Installing the ``larray-editor`` package on Windows will create a ``LArray`` menu in the
Windows Start Menu. This menu contains:

  * a shortcut to open the documentation of the last stable version of the library
  * a shortcut to open the graphical interface in edit mode.
  * a shortcut to update `larrayenv`.

![menu_windows](../_static/menu_windows.png)

![editor_new](../_static/editor_new.png)

Once the graphical interface is open, all LArray objects and functions are directly accessible.
No need to start by `from larray import *`.